In [4]:
%load_ext autoreload

In [5]:
%autoreload 2

In [10]:
from pathlib import Path

In [11]:
from smtag.config import config
config

Config(max_length=512, truncation=True, min_char_length=120, celery_batch_size=1000, from_pretrained='roberta-base', model_type='Autoencoder', nlp=<spacy.lang.en.English object at 0x7fd4142b98b0>)

In [12]:
from transformers import __version__
__version__

'4.15.0'

## Download SourceData data

This takes a very long time! It advised to run this within a tmux session rather than in this notebook.

In [ ]:
! python -m smtag.cli.prepo.get_sd  # files saved in xml_destination_files
! # mv xml_destination_files/ data/xml/<name_of_source_data_compendium>

## Split dataset

In [ ]:
from smtag.split import distribute

In [ ]:
distribute(Path("data/xml/220304_sd"), ext="xml")

## Extracting examples for TOKCL

In [9]:
from smtag.extract import ExtractorXML

#### Dataset with individual panels

In [10]:
xml_panel_examples = "/data/text/220304_sd_panels"

In [11]:
! rm -fr /data/text/220304_panels_sd

In [12]:
extractor_tokcl = ExtractorXML(
    "/data/xml/220304_sd",
    destination_dir=xml_panel_examples,
    sentence_level=False,
    xpath=".//sd-panel",
    keep_xml=True,
    inclusion_probability=1.0
)

/data/text/220304_sd_panels created


In [13]:
extractor_tokcl.extract_from_corpus()

100%|██████████| 1/1 [00:10<00:00, 10.43s/it]


{PosixPath('/data/text/220304_sd_panels/train.txt'): 48771,
 PosixPath('/data/text/220304_sd_panels/eval.txt'): 13801,
 PosixPath('/data/text/220304_sd_panels/test.txt'): 7178}

same via CLI:

```bash
python -m smtag.cli.prepro.extract /data/xml/191012/ /data/text/sd_test --xpath ".//sd-panel" --sentence_level --keep_xml --inclusion_probability 1.0
```

#### Dataset with full figures (used for panelization training)

In [16]:
xml_figure_examples = "/data/text/220304_sd_fig"

In [17]:
! rm -fr "/data/text/220304_sd_fig"

In [18]:
extractor_tokcl_2 = ExtractorXML(
    "/data/xml/220304_sd",
    destination_dir=xml_figure_examples,
    sentence_level=False,
    xpath=".//fig",
    keep_xml=True,
    inclusion_probability=1.0
)

/data/text/220304_sd_fig created


In [19]:
extractor_tokcl_2.extract_from_corpus()

100%|██████████| 1/1 [00:03<00:00,  3.31s/it]


{PosixPath('/data/text/220304_sd_fig/train.txt'): 12108,
 PosixPath('/data/text/220304_sd_fig/eval.txt'): 3439,
 PosixPath('/data/text/220304_sd_fig/test.txt'): 1802}

## Preparing tokenized dataset for TOKCL

In [20]:
from smtag.dataprep import PreparatorTOKCL

In [21]:
from smtag.xml2labels import SourceDataCodes as sd

#### Tokenize panels

In [22]:
! rm -fr /data/json/220304_sd_panels

In [23]:
tokenized_panel_examples = "/data/json/220304_sd_panels"
code_maps: code_maps = [
    sd.ENTITY_TYPES,
    sd.GENEPROD_ROLES,
    sd.SMALL_MOL_ROLES,
    sd.BORING,
    sd.PANELIZATION
]

In [24]:
prep_tokcl = PreparatorTOKCL(
    xml_panel_examples,
    tokenized_panel_examples,
    code_maps,
    max_length=config.max_length
)

/data/json/220304_sd_panels created


In [25]:
prep_tokcl.run()

Preparing: train


  7%|▋         | 3377/48771 [00:07<01:36, 472.07it/s]

 11%|█         | 5462/48771 [00:12<01:44, 414.72it/s]

 11%|█▏        | 5585/48771 [00:12<01:56, 369.58it/s]

 21%|██        | 10105/48771 [00:22<01:22, 467.36it/s]

...

100%|██████████| 13801/13801 [00:32<00:00, 431.00it/s]



Length verification: OK!

average input_ids length = 101 (min=22, max=512) tokens
longest example: <s> A1) Representative WBs showing DRD2 and RGS9-2 downregulation in adult (P60-P90) Tor1a+/- (+/-) striata, characterized by a reduced torsinA protein level with respect to Tor1a+/+ (+/+) littermates. The striatal level of RGS7 is unchanged. The dot plot shows Tor1a+/- data, normalized to Tor1a+/+ controls of the same experiment. TorsinA: Tor1a+/+ N=7, Tor1a+/- N=6, t test ***P=0.0003; DRD2: Tor1a+/+ N=18, Tor1a+/- N=16, t test **P=0.0029; RGS9-2: Tor1a+/+ N=32, Tor1a+/- N=33, t test *P=0.0115). RGS7: Tor1a+/+ N=7, Tor1a+/- N=7, t test P=0.8721). WB quantification data, expressed as the ratio of protein vs. loading control intensity level, are normalized to the wild-type samples of the same experiment. A2,A3,A4) Time-course of changes in torsinA, DRD2 and RGS9-2 striatal levels along postnatal development. Summary plot data are normalized to the Tor1a+/+ P60 sample of each independent e

 18%|█▊        | 1301/7178 [00:03<00:13, 433.86it/s]

100%|██████████| 7178/7178 [00:16<00:00, 424.51it/s]



Length verification: OK!

average input_ids length = 103 (min=25, max=512) tokens
longest example: <s>Either young WT (A, B) or old APPPS1 (C-E) tissue was treated with clodronate (Clo) to remove CD68 positive cells or vehicle Ctr from 1 until 7 DIV. Treatment was stopped and subsequently old (A, B) or young (C-E) tissue was added to the culture as schematically indicated in A and C. and analyzed 14 days after. Removal of CD68 positive cells in either young WT or old APPPS1 tissue prevents amyloid plaque clearance in the co-culture model.(A) Immunofluorescence analysis of the old APPPS1 slice co-cultured with the young WT slice pre-treated with Clo and Ctr and immunostained with CD68 (red) and M3.2 (green). Scale bar: 50 µm.(B) Quantitative analysis of core-only plaques in the old APPPS1 tissue co-cultured with the young WT tissue pre-treated with Clo and Ctr as indicated in A reveals a decreased number of core-only plaques upon Clo treatment. The values are expressed as percentages o

same vie CLI:
    
```bash
python -m smtag.cli.tokcl.dataprep /data/text/sd_test /data/json/sd_test
```

#### Tokenize figures

In [26]:
tokenized_figure_examples = "/data/json/220304_sd_fig"
code_maps: code_maps = [
    sd.PANELIZATION
]

In [27]:
! rm -fr /data/json/220304_sd_fig

In [28]:
prep_tokcl_2 = PreparatorTOKCL(
    xml_figure_examples,
    tokenized_figure_examples,
    code_maps,
    max_length=config.max_length
)

/data/json/220304_sd_fig created


In [29]:
prep_tokcl_2.run()

Preparing: train


 24%|██▍       | 2961/12108 [00:08<00:25, 361.41it/s]

 34%|███▍      | 4145/12108 [00:11<00:24, 320.91it/s]

 55%|█████▌    | 6674/12108 [00:19<00:14, 365.14it/s]

100%|██████████| 12108/12108 [00:35<00:00, 338.58it/s]


...

 50%|████▉     | 1714/3439 [00:05<00:04, 371.60it/s]

100%|██████████| 3439/3439 [00:10<00:00, 321.86it/s]



Length verification: OK!

average input_ids length = 367 (min=26, max=512) tokens
longest example: <s>.Figure 4(A) Cryo-electron tomogram of P. tetraurelia centriole. Blue arrow denotes cartwheel, orange arrow denotes inner scaffold. White dashed box delimits the inset represented in (D, E). Red lines and arrows indicate the position and the direction of the cross-sections made in (B). Scale bar, 100 nm. (B) Nine-fold symmetrizations of serial cross-sections taken along the proximal to distal axis in P. tetraurelia. Each section is a z-projection of 20.7 nm. White dashed circles delineate the structures highlighted in C. Scale bar, 60 nm. (C) Zoomed images of proximal centriole substructures from nine-fold symmetrizations of P. tetraurelia along the proximal-distal axis. Each panel corresponds to the above image from panel B. Purple arrow, pinhead; light green arrow, triplet base; turquoise arrow, A-C linker; orange arrow, inner scaffold. (D) Side view showing the transition from pinh

100%|██████████| 1802/1802 [00:05<00:00, 323.53it/s]



Length verification: OK!

average input_ids length = 369 (min=30, max=512) tokens
longest example: <s>: eIF4A is required for complete TORC1 inactivation upon amino acid removal.Figure 1(a-b) eIF4A knockdown in either S2 (a) or Kc167 (b) cells blunts the inactivation of TORC1 upon amino acid removal. Cells were treated with dsRNA targeting GFP as a negative control, or three independent, non-overlapping dsRNAs targeting eIF4A for 5 days and then incubated with complete medium or medium lacking only amino acids for 30 minutes. Representative of 3 biological replicates. (c) Only knockdown of eIF4A, but not of other genes involved in translation, causes elevated TORC1 upon amino acid withdrawal. S2 cells were treated with dsRNAs for 5 days and then incubated with medium containing or lacking amino acids for 30 minutes. After cell lysis, levels of S6K phosphorylation (T398) were measured by dot blot analysis and normalized to total S6K levels. Results are summarized here for all genes inv

## Train model for TOKCL

In [30]:
from smtag.config import config
from smtag.train.train_tokcl import (
    train as train_tokcl,
    TrainingArgumentsTOKCL
)

In [31]:
training_args = TrainingArgumentsTOKCL(
    logging_steps=50,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
)

In [13]:
loader_path = "./smtag/loader/loader_tokcl.py"
tokenizer = config.tokenizer
model_type = "Autoencoder"
from_pretrained = "EMBO/bio-lm"  # "roberta-base" # specialized model from huggingface.co/embo #  "roberta-base" # general lm model

Reload the datasets afresh (to prevent this behavior, set `no_cache` to `False`):

In [33]:
no_cache = True

In [52]:
! rm -fr /runs/*

In [53]:
! rm -fr /tokcl_models/*

### Train NER

In [54]:
training_args.overwrite_output_dir=False

In [55]:
training_args.num_train_epochs=0.6
training_args.prediction_loss_only=True
training_args.masking_probability=.0
training_args.replacement_probability=.0
training_args

TrainingArgumentsTOKCL(output_dir='/tokcl_models', overwrite_output_dir=False, do_train=False, do_eval=True, do_predict=False, evaluation_strategy=<IntervalStrategy.STEPS: 'steps'>, prediction_loss_only=True, per_device_train_batch_size=16, per_device_eval_batch_size=16, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=0.0001, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=0.6, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_ratio=0.0, warmup_steps=0, log_level=-1, log_level_replica=-1, log_on_each_node=True, logging_dir='/tokcl_models/runs/Mar06_14-19-46_5aebd0ba4077', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=20, logging_nan_inf_filter=True, save_strategy=<IntervalStrategy.STEPS: 'steps'>, save_steps=500, save_total_limit=5, save_on_each_node=False, no_cuda=False, 

In [56]:
train_tokcl(
    training_args,
    loader_path,
    "NER",
    tokenized_panel_examples,
    no_cache,
    tokenizer,
    model_type,
    from_pretrained
)

Created /tokcl_models/NER.
tokenizer vocab size: 50265

Loading and tokenizing datasets found in /data/json/220304_sd_panels.
using ./smtag/loader/loader_tokcl.py as dataset loader.


0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset source_data_nlp downloaded and prepared to /cache/source_data_nlp/NER-ff39ee0af78a5a25/0.0.1/be20c29bc25cb738f99d32581855842b36617d57e5a36ca1d6b98499db149e83. Subsequent calls will reuse this data.


...

***** Running Evaluation *****
  Num examples = 13801
  Batch size = 64




<s>Micrograph of coronal sections of E17 mouse cerebral cortices electroporated at E14 co-electroporated with mCherry expressing vector and HA-Cstb or R68X, and analyzed 3 dpe. Immunostaining with RFP to identify electroporated cells and GFP to identify migrating interneurons in the GAD65-GFP transgenic mouse line. Ventricle (V) is indicated. The dashed lines represent the apical surface of the ventricles. Quantification of the total number of GAD65-GFP interneurons/ cortical area (��m2) in (N). Data shown as Z-scores relative to the mean of GFP control vector. Nuclei (blue) are stained with DAPI. </s>






Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /tokcl_models/NER
Configuration saved in /tokcl_models/NER/config.json
Model weights saved in /tokcl_models/NER/pytorch_model.bin
***** Running Prediction *****
  Num examples = 7178
  Batch size = 64


Testing on 7178.



                                                                                
                precision    recall  f1-score   support

          CELL       0.69      0.81      0.74      5245
     EXP_ASSAY       0.56      0.57      0.56     10067
      GENEPROD       0.77      0.89      0.82     23587
      ORGANISM       0.72      0.82      0.77      3623
SMALL_MOLECULE       0.70      0.80      0.75      6187
   SUBCELLULAR       0.65      0.72      0.69      3700
        TISSUE       0.62      0.73      0.67      3207

     micro avg       0.70      0.79      0.74     55616
     macro avg       0.67      0.77      0.72     55616
  weighted avg       0.70      0.79      0.74     55616

{'test_loss': 0.1830928772687912, 'test_accuracy_score': 0.9334821000160841, 'test_precision': 0.6987463009514112, 'test_recall': 0.789682825086306, 'test_f1': 0.7414366506288511, 'test_runtime': 61.0547, 'test_samples_per_second': 117.567, 'test_steps_per_second': 1.851}


#### Train GENEPROD ROLES

In [57]:
training_args.num_train_epochs = 0.9
training_args.prediction_loss_only=True
training_args.masking_probability=1.
training_args.replacement_probability=.0
training_args

TrainingArgumentsTOKCL(output_dir='/tokcl_models', overwrite_output_dir=False, do_train=False, do_eval=True, do_predict=False, evaluation_strategy=<IntervalStrategy.STEPS: 'steps'>, prediction_loss_only=True, per_device_train_batch_size=16, per_device_eval_batch_size=16, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=0.0001, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=0.9, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_ratio=0.0, warmup_steps=0, log_level=-1, log_level_replica=-1, log_on_each_node=True, logging_dir='/tokcl_models/runs/Mar06_14-19-46_5aebd0ba4077', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=20, logging_nan_inf_filter=True, save_strategy=<IntervalStrategy.STEPS: 'steps'>, save_steps=500, save_total_limit=5, save_on_each_node=False, no_cuda=False, 

In [58]:
training_args.output_dir

'/tokcl_models'

In [59]:
train_tokcl(
    training_args,
    loader_path,
    "GENEPROD_ROLES",
    tokenized_panel_examples,
    no_cache,
    tokenizer,
    model_type,
    from_pretrained
)

Created /tokcl_models/GENEPROD_ROLES.
tokenizer vocab size: 50265

Loading and tokenizing datasets found in /data/json/220304_sd_panels.
using ./smtag/loader/loader_tokcl.py as dataset loader.


0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset source_data_nlp downloaded and prepared to /cache/source_data_nlp/GENEPROD_ROLES-ff39ee0af78a5a25/0.0.1/be20c29bc25cb738f99d32581855842b36617d57e5a36ca1d6b98499db149e83. Subsequent calls will reuse this data.


...

***** Running Evaluation *****
  Num examples = 13801
  Batch size = 64




<s>We suggest that H3.3-containing nucleosome cores (pink) are more likely to be dynamically exchanged than canonical H3-containing cores (blue) resulting in a greater probability that DNA can become single stranded either as a result of pausing of RNAPII or of transcription-induced negative supercoiling generating transient denaturation bubbles. </s>






Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /tokcl_models/GENEPROD_ROLES
Configuration saved in /tokcl_models/GENEPROD_ROLES/config.json
Model weights saved in /tokcl_models/GENEPROD_ROLES/pytorch_model.bin
***** Running Prediction *****
  Num examples = 7178
  Batch size = 64


Testing on 7178.



                                                                                
                precision    recall  f1-score   support

CONTROLLED_VAR       0.81      0.86      0.83      7835
  MEASURED_VAR       0.82      0.85      0.84      9330

     micro avg       0.82      0.85      0.83     17165
     macro avg       0.82      0.85      0.83     17165
  weighted avg       0.82      0.85      0.83     17165

{'test_loss': 0.03846803680062294, 'test_accuracy_score': 0.9854472664459946, 'test_precision': 0.8156312625250501, 'test_recall': 0.8535974366443344, 'test_f1': 0.8341825841897008, 'test_runtime': 58.7369, 'test_samples_per_second': 122.206, 'test_steps_per_second': 1.924}


### Train SMALL MOL ROLES

In [60]:
training_args.num_train_epochs = 0.33
training_args.prediction_loss_only=True
training_args.masking_probability=1.0
training_args.replacement_probability=.0
training_args

TrainingArgumentsTOKCL(output_dir='/tokcl_models', overwrite_output_dir=False, do_train=False, do_eval=True, do_predict=False, evaluation_strategy=<IntervalStrategy.STEPS: 'steps'>, prediction_loss_only=True, per_device_train_batch_size=16, per_device_eval_batch_size=16, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=0.0001, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=0.33, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_ratio=0.0, warmup_steps=0, log_level=-1, log_level_replica=-1, log_on_each_node=True, logging_dir='/tokcl_models/runs/Mar06_14-19-46_5aebd0ba4077', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=20, logging_nan_inf_filter=True, save_strategy=<IntervalStrategy.STEPS: 'steps'>, save_steps=500, save_total_limit=5, save_on_each_node=False, no_cuda=False,

In [61]:
train_tokcl(
    training_args,
    loader_path,
    "SMALL_MOL_ROLES",
    tokenized_panel_examples,
    no_cache,
    tokenizer,
    model_type,
    from_pretrained
)

/opt/conda/lib/python3.8/site-packages/datasets/load.py:1650: FutureWarning: 'script_version' was renamed to 'revision' in version 1.13 and will be removed in 1.15.
  warnings.warn(


Created /tokcl_models/SMALL_MOL_ROLES.
tokenizer vocab size: 50265

Loading and tokenizing datasets found in /data/json/220304_sd_panels.
using ./smtag/loader/loader_tokcl.py as dataset loader.


0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

...

***** Running Evaluation *****
  Num examples = 13801
  Batch size = 64




<s>(D) The spine head width distribution7differed significantly between the genotypes (n= 467 spines from WT and n = 421 spines from CA7 KO animals, 15 neurons analyzed from both genotypes, Wilcoxon rank sum test with continuity correction, W = 134540, P < 0.001). </s>






Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /tokcl_models/SMALL_MOL_ROLES
Configuration saved in /tokcl_models/SMALL_MOL_ROLES/config.json
Model weights saved in /tokcl_models/SMALL_MOL_ROLES/pytorch_model.bin
***** Running Prediction *****
  Num examples = 7178
  Batch size = 64


Testing on 7178.



                                                                                
                precision    recall  f1-score   support

CONTROLLED_VAR       0.76      0.90      0.83      2946
  MEASURED_VAR       0.60      0.71      0.65       852

     micro avg       0.73      0.86      0.79      3798
     macro avg       0.68      0.80      0.74      3798
  weighted avg       0.73      0.86      0.79      3798

{'test_loss': 0.011743436567485332, 'test_accuracy_score': 0.9951612532624371, 'test_precision': 0.7261345852895149, 'test_recall': 0.8551869404949973, 'test_f1': 0.7853947527505744, 'test_runtime': 58.0378, 'test_samples_per_second': 123.678, 'test_steps_per_second': 1.947}


### Train PANELIZATION

In [62]:
training_args.num_train_epochs = 1.3
training_args.prediction_loss_only=True
training_args.masking_probability=.0
training_args.replacement_probability=.0
training_args.logging_steps=20
training_args

TrainingArgumentsTOKCL(output_dir='/tokcl_models', overwrite_output_dir=False, do_train=False, do_eval=True, do_predict=False, evaluation_strategy=<IntervalStrategy.STEPS: 'steps'>, prediction_loss_only=True, per_device_train_batch_size=16, per_device_eval_batch_size=16, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=0.0001, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.3, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_ratio=0.0, warmup_steps=0, log_level=-1, log_level_replica=-1, log_on_each_node=True, logging_dir='/tokcl_models/runs/Mar06_14-19-46_5aebd0ba4077', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=20, logging_nan_inf_filter=True, save_strategy=<IntervalStrategy.STEPS: 'steps'>, save_steps=500, save_total_limit=5, save_on_each_node=False, no_cuda=False, 

In [63]:
train_tokcl(
    training_args,
    loader_path,
    "PANELIZATION",
    tokenized_figure_examples,  # Use Figure-level data here!
    no_cache,
    tokenizer,
    model_type,
    from_pretrained
)

Created /tokcl_models/PANELIZATION.
tokenizer vocab size: 50265

Loading and tokenizing datasets found in /data/json/220304_sd_fig.
using ./smtag/loader/loader_tokcl.py as dataset loader.


0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset source_data_nlp downloaded and prepared to /cache/source_data_nlp/PANELIZATION-3c1c867cbe013e60/0.0.1/be20c29bc25cb738f99d32581855842b36617d57e5a36ca1d6b98499db149e83. Subsequent calls will reuse this data.


...

***** Running Evaluation *****
  Num examples = 3439
  Batch size = 64




<s>.Figure 5A. Fold change (FC) and significance for transcripts in Kdm6apKO vs. control pancreas. Genes significant at q<0.05 are shown as colored dots above the horizontal line. B. GSEA showing that genes specific to differentiated acinar cells were downregulated in KDM6A-deficient pancreas, but not genes specific to islets or duct cells. Upregulated genes were enriched in genes specific to mesenchymal cells. Lineage-enriched genes were obtained from (Muraro et al., 2016). C. Most enriched functional annotations in genes differentially expressed in Kdm6apKO. D. GSEA showed increased expression of indicated oncogenic pathway genes in Kdm6apKO pancreas. E. KDM6A functions as a transcriptional activator of direct target genes in pancreatic cells. Left: Downregulated, but not upregulated, genes in Kdm6apKO pancreas were enriched for KDM6A binding. P-values and odds ratios (O.R.) were calculated by Fisher's exact test. Right: Venn diagrams showing overlap of down and upregulated genes w



Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /tokcl_models/PANELIZATION
Configuration saved in /tokcl_models/PANELIZATION/config.json
Model weights saved in /tokcl_models/PANELIZATION/pytorch_model.bin
***** Running Prediction *****
  Num examples = 1802
  Batch size = 64


Testing on 1802.



                                                                                
              precision    recall  f1-score   support

 PANEL_START       0.89      0.95      0.92      5427

   micro avg       0.89      0.95      0.92      5427
   macro avg       0.89      0.95      0.92      5427
weighted avg       0.89      0.95      0.92      5427

{'test_loss': 0.0037643597461283207, 'test_accuracy_score': 0.9986864826439872, 'test_precision': 0.8946091176980413, 'test_recall': 0.9509858116823291, 'test_f1': 0.9219364058592354, 'test_runtime': 33.4943, 'test_samples_per_second': 53.8, 'test_steps_per_second': 0.866}


In [48]:
! ls /tokcl_models/

GENEPROD_ROLES	NER  PANELIZATION  SMALL_MOL_ROLES


### Alternative via CLI:

Useful for testing and debugging from within `tmux` session and `docker-compose exec nlp bash`

```bash
python -m smtag.cli.tokcl.train \
./smtag/loader/loader_tokcl.py \
PANELIZATION \
--data_dir /data/json/sd_test \
--num_train_epochs=1 \
--logging_steps=50 \
--per_device_train_batch_size=16 \
--per_device_eval_batch_size=16 \
--replacement_probability=0 \
--masking_probability=0 \
--model_type=Autoencoder \
--from_pretrained="EMBO/bio-lm"
```

## Try it!

In [1]:
from smtag.pipeline import SmartTagger

In [2]:
smarttagger = SmartTagger(
    tokenizer_source="roberta-base",
    panelizer_source="/tokcl_models/PANELIZATION",
    ner_source="/tokcl_models/NER",
    geneprod_roles_source="/tokcl_models/GENEPROD_ROLES",
    small_mol_roles_source="/tokcl_models/SMALL_MOL_ROLES"
)

In [3]:
tags = smarttagger("This creb1-/- mutant mouse has a strange brain after aspirin treatment.")
print(tags)

{
  "smtag": [
    {
      "panel_group": [
        [
          {
            "text": "creb1",
            "entity_type": "geneprod"
          },
          {
            "text": "mouse",
            "entity_type": "organism"
          },
          {
            "text": "brain",
            "entity_type": "tissue"
          },
          {
            "text": "aspirin",
            "entity_type": "molecule",
            "role": "intervention"
          }
        ]
      ]
    }
  ]
}


With CLI:

    python -m smtag.cli.inference.tag --local_model_dir /tokcl_models "This creb1-/- mutant mouse has a strange brain after aspirin treatment."

# Before sharing

Save the tokenizer in the respective model directories to enable inference 

In [20]:
tokenizer.save_pretrained("/tokcl_models/NER")
tokenizer.save_pretrained("/tokcl_models/GENEPROD_ROLES")
tokenizer.save_pretrained("/tokcl_models/SMALL_MOL_ROLES")
tokenizer.save_pretrained("/tokcl_models/PANELIZATION")

('/tokcl_models/PANELIZATION/tokenizer_config.json',
 '/tokcl_models/PANELIZATION/special_tokens_map.json',
 '/tokcl_models/PANELIZATION/vocab.json',
 '/tokcl_models/PANELIZATION/merges.txt',
 '/tokcl_models/PANELIZATION/added_tokens.json',
 '/tokcl_models/PANELIZATION/tokenizer.json')

In [19]:
!ls tokcl_models/NER

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
config.json	   special_tokens_map.json  tokenizer_config.json
merges.txt	   tokenizer		    training_args.bin
pytorch_model.bin  tokenizer.json	    vocab.json


In [14]:
dir(tokenizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_tokens',
 '_additional_special_tokens',
 '_batch_encode_plus',
 '_bos_token',
 '_build_conversation_input_ids',
 '_cls_token',
 '_convert_encoding',
 '_convert_id_to_token',
 '_convert_token_to_id_with_added_voc',
 '_create_or_get_repo',
 '_decode',
 '_decode_use_source_tokenizer',
 '_encode_plus',
 '_eos_token',
 '_eventual_warn_about_too_long_sequence',
 '_from_pretrained',
 '_get_padding_truncation_strategies',
 '_get_repo_url_from_name',
 '_mask_token',
 '_pad',
 '_pad_token',
 '_pad_token_type_id',
 '_push_to_hub',
 '_save_pretrained',
 